# Step 9 — Telemetry Module

**What we built**: A `TelemetryModule` that logs structured timing, token usage, and cost per `invoke()` call.

**Why it matters**: In production with thousands of agents, operators need visibility into every LLM call — how long it took, how many tokens were used, and how much it cost. Telemetry sits *outermost* in the module stack, measuring total wall-clock duration including retries, fallback attempts, and rate-limit waits.

**Key decisions**:
- **D-064**: Structured logging only — no callback, no accumulator (simple, toggle-able)
- **D-065**: Calculate and log `cost_usd` per call from provider pricing metadata
- **D-066**: `load_model()` injects pricing from provider TOML via `setdefault()`
- **D-067**: Outermost position: `Telemetry(Retry(Fallback(RateLimit(adapter))))`
- **D-068**: INFO by default, configurable via `log_level`
- **D-069**: Fields: provider, model, duration_ms, tokens, cache (conditional), cost_usd, stop_reason

**Stack position**: `Telemetry → Retry → Fallback → RateLimit → Adapter`

In [ ]:
# Setup: ensure arcllm is importable
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

---
## 1. What Gets Logged

Every `invoke()` call produces one structured log line:

```
INFO  LLM call | provider=anthropic model=claude-sonnet-4-20250514 duration_ms=1234.5
               input_tokens=100 output_tokens=50 total_tokens=150
               cache_read_tokens=80 cache_write_tokens=20
               cost_usd=0.001149 stop_reason=end_turn
```

**Always present**: provider, model, duration_ms, input/output/total tokens, cost_usd, stop_reason

**Conditional**: cache_read_tokens, cache_write_tokens (omitted when None — reduces log noise for providers that don't use caching)

---
## 2. Constructing TelemetryModule

In [ ]:
from unittest.mock import AsyncMock, MagicMock
from arcllm.modules.telemetry import TelemetryModule
from arcllm.types import LLMProvider, LLMResponse, Message, Usage

# Create a mock inner adapter
inner = MagicMock(spec=LLMProvider)
inner.name = "anthropic"
inner.model_name = "claude-sonnet-4-20250514"
inner.invoke = AsyncMock(return_value=LLMResponse(
    content="Hello!",
    usage=Usage(input_tokens=100, output_tokens=50, total_tokens=150),
    model="claude-sonnet-4-20250514",
    stop_reason="end_turn",
))

# Config with Anthropic Sonnet pricing
config = {
    "cost_input_per_1m": 3.00,
    "cost_output_per_1m": 15.00,
    "cost_cache_read_per_1m": 0.30,
    "cost_cache_write_per_1m": 3.75,
}

module = TelemetryModule(config, inner)

print(f"Module type:      {type(module).__name__}")
print(f"Provider (inner): {module.name}")
print(f"Model (inner):    {module.model_name}")
print(f"Cost input/1M:    ${module._cost_input:.2f}")
print(f"Cost output/1M:   ${module._cost_output:.2f}")
print(f"Cost cache rd/1M: ${module._cost_cache_read:.2f}")
print(f"Cost cache wr/1M: ${module._cost_cache_write:.2f}")
print(f"Log level:        {module._log_level} (10=DEBUG, 20=INFO)")

---
## 3. Cost Calculation

Formula: `cost_usd = (input_tokens × cost_input / 1M) + (output_tokens × cost_output / 1M) + cache costs`

In [ ]:
# Basic cost: input + output only
usage_basic = Usage(input_tokens=1000, output_tokens=500, total_tokens=1500)
cost = module._calculate_cost(usage_basic)
print(f"Basic cost (1K in, 500 out):")
print(f"  Input:  1000 × $3.00 / 1M = ${1000 * 3.00 / 1_000_000:.6f}")
print(f"  Output: 500 × $15.00 / 1M = ${500 * 15.00 / 1_000_000:.6f}")
print(f"  Total:  ${cost:.6f}")

In [ ]:
# Cost with all 4 token types
usage_full = Usage(
    input_tokens=1000,
    output_tokens=500,
    total_tokens=1500,
    cache_read_tokens=800,
    cache_write_tokens=200,
)
cost_full = module._calculate_cost(usage_full)
print(f"Full cost (with cache):")
print(f"  Input:       1000 × $3.00 / 1M = ${1000 * 3.00 / 1_000_000:.6f}")
print(f"  Output:       500 × $15.00 / 1M = ${500 * 15.00 / 1_000_000:.6f}")
print(f"  Cache read:   800 × $0.30 / 1M = ${800 * 0.30 / 1_000_000:.6f}")
print(f"  Cache write:  200 × $3.75 / 1M = ${200 * 3.75 / 1_000_000:.6f}")
print(f"  Total: ${cost_full:.6f}")

In [ ]:
# At exactly 1M tokens — cost should match the per-1M prices
usage_1m = Usage(input_tokens=1_000_000, output_tokens=1_000_000, total_tokens=2_000_000)
cost_1m = module._calculate_cost(usage_1m)
print(f"At 1M tokens each: ${cost_1m:.2f} (= $3.00 input + $15.00 output)")

In [ ]:
# Zero pricing → zero cost (module created with empty config)
module_free = TelemetryModule({}, inner)
cost_free = module_free._calculate_cost(usage_basic)
print(f"No pricing configured: ${cost_free:.6f}")
print(f"All cost fields default to 0.0: input={module_free._cost_input}, output={module_free._cost_output}")

---
## 4. Timing with time.monotonic()

Wall-clock duration is measured around the entire `inner.invoke()` call, including any retries/fallback/rate-limit waits that happen below.

In [ ]:
import logging
from unittest.mock import patch

# Mock time to control exact duration
with patch("arcllm.modules.telemetry.time.monotonic") as mock_mono:
    mock_mono.side_effect = [1000.0, 1000.5]  # 500ms elapsed
    
    inner = MagicMock(spec=LLMProvider)
    inner.name = "anthropic"
    inner.invoke = AsyncMock(return_value=LLMResponse(
        content="ok",
        usage=Usage(input_tokens=100, output_tokens=50, total_tokens=150),
        model="claude-sonnet-4-20250514",
        stop_reason="end_turn",
    ))
    
    module = TelemetryModule(config, inner)
    
    # Capture log output
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    tel_logger = logging.getLogger("arcllm.modules.telemetry")
    tel_logger.addHandler(handler)
    tel_logger.setLevel(logging.INFO)
    
    result = await module.invoke([Message(role="user", content="hi")])
    
    tel_logger.removeHandler(handler)

print(f"\nduration_ms = round(0.5 * 1000, 1) = 500.0")

The timing code:
```python
start = time.monotonic()
response = await self._inner.invoke(messages, tools, **kwargs)
elapsed = time.monotonic() - start
duration_ms = round(elapsed * 1000, 1)
```

`time.monotonic()` — immune to system clock adjustments (NTP corrections, DST changes).

---
## 5. Conditional Cache Token Fields

Cache tokens are only logged when present (not None). This reduces log noise for providers/calls that don't use prompt caching.

In [ ]:
# Response WITHOUT cache tokens
resp_no_cache = LLMResponse(
    content="ok",
    usage=Usage(input_tokens=100, output_tokens=50, total_tokens=150),
    model="test",
    stop_reason="end_turn",
)

# Response WITH cache tokens
resp_cached = LLMResponse(
    content="cached",
    usage=Usage(
        input_tokens=100, output_tokens=50, total_tokens=150,
        cache_read_tokens=80, cache_write_tokens=20,
    ),
    model="test",
    stop_reason="end_turn",
)

print(f"No cache - cache_read_tokens: {resp_no_cache.usage.cache_read_tokens}")
print(f"No cache - cache_write_tokens: {resp_no_cache.usage.cache_write_tokens}")
print(f"Cached - cache_read_tokens: {resp_cached.usage.cache_read_tokens}")
print(f"Cached - cache_write_tokens: {resp_cached.usage.cache_write_tokens}")
print("\nWhen None → fields omitted from log line")
print("When present → fields included")

In [ ]:
# Demonstrate the conditional logic
import logging

# Test 1: No cache fields in log
with patch("arcllm.modules.telemetry.time.monotonic", side_effect=[0.0, 0.1]):
    inner1 = MagicMock(spec=LLMProvider)
    inner1.name = "test"
    inner1.invoke = AsyncMock(return_value=resp_no_cache)
    mod = TelemetryModule(config, inner1)
    
    handler = logging.StreamHandler()
    tel_logger = logging.getLogger("arcllm.modules.telemetry")
    tel_logger.addHandler(handler)
    tel_logger.setLevel(logging.INFO)
    
    print("=== Without cache tokens ===")
    await mod.invoke([Message(role="user", content="hi")])
    print("(no cache_read_tokens or cache_write_tokens in output)")
    
    tel_logger.removeHandler(handler)

# Test 2: Cache fields present
with patch("arcllm.modules.telemetry.time.monotonic", side_effect=[0.0, 0.1]):
    inner2 = MagicMock(spec=LLMProvider)
    inner2.name = "test"
    inner2.invoke = AsyncMock(return_value=resp_cached)
    mod2 = TelemetryModule(config, inner2)
    
    tel_logger.addHandler(handler)
    
    print("\n=== With cache tokens ===")
    await mod2.invoke([Message(role="user", content="hi")])
    print("(cache_read_tokens=80 and cache_write_tokens=20 included)")
    
    tel_logger.removeHandler(handler)

---
## 6. Configurable Log Level

In [ ]:
from arcllm.modules.telemetry import _VALID_LOG_LEVELS

print(f"Valid log levels: {sorted(_VALID_LOG_LEVELS)}")
print(f"Default: INFO")
print()

# DEBUG level — won't show at INFO
debug_config = {**config, "log_level": "DEBUG"}
inner_d = MagicMock(spec=LLMProvider)
inner_d.name = "test"
inner_d.invoke = AsyncMock(return_value=resp_no_cache)

mod_debug = TelemetryModule(debug_config, inner_d)
print(f"Module log level: {mod_debug._log_level} (10=DEBUG)")
print("At INFO capture level, DEBUG messages are invisible")
print("At DEBUG capture level, they appear")

---
## 7. Config Validation

In [ ]:
from arcllm.exceptions import ArcLLMConfigError

inner = MagicMock(spec=LLMProvider)
inner.name = "test"

# Negative input cost → rejected
try:
    TelemetryModule({"cost_input_per_1m": -1.0}, inner)
except ArcLLMConfigError as e:
    print(f"Negative input cost:  {e}")

# Negative output cost → rejected
try:
    TelemetryModule({"cost_output_per_1m": -1.0}, inner)
except ArcLLMConfigError as e:
    print(f"Negative output cost: {e}")

# Invalid log level → rejected
try:
    TelemetryModule({"log_level": "INVALID"}, inner)
except ArcLLMConfigError as e:
    print(f"Invalid log level:    {e}")

# Empty config → all defaults to 0.0 (valid)
mod_empty = TelemetryModule({}, inner)
print(f"\nEmpty config (all zeros): input={mod_empty._cost_input}, output={mod_empty._cost_output}")

---
## 8. Response Passthrough

TelemetryModule returns the **exact same response object** — no copies, no modifications.

In [ ]:
with patch("arcllm.modules.telemetry.time.monotonic", side_effect=[0.0, 0.1]):
    original_response = LLMResponse(
        content="original",
        usage=Usage(input_tokens=10, output_tokens=5, total_tokens=15),
        model="test",
        stop_reason="end_turn",
    )
    
    inner = MagicMock(spec=LLMProvider)
    inner.name = "test"
    inner.invoke = AsyncMock(return_value=original_response)
    mod = TelemetryModule(config, inner)
    
    result = await mod.invoke([Message(role="user", content="hi")])
    
    # Same object (is check, not equality)
    print(f"Same object? {result is original_response}")
    print(f"Content: {result.content}")

---
## 9. Pricing Injection via load_model()

The elegant part: pricing data lives in provider TOML files, and `load_model()` bridges it to TelemetryModule via `setdefault()`.

In [ ]:
# What's in anthropic.toml:
from arcllm.config import load_provider_config

provider_config = load_provider_config("anthropic")
sonnet_meta = provider_config.models["claude-sonnet-4-20250514"]
haiku_meta = provider_config.models["claude-haiku-4-5-20251001"]

print("=== Sonnet Pricing (from anthropic.toml) ===")
print(f"  Input:       ${sonnet_meta.cost_input_per_1m}/1M")
print(f"  Output:      ${sonnet_meta.cost_output_per_1m}/1M")
print(f"  Cache read:  ${sonnet_meta.cost_cache_read_per_1m}/1M")
print(f"  Cache write: ${sonnet_meta.cost_cache_write_per_1m}/1M")

print("\n=== Haiku Pricing ===")
print(f"  Input:       ${haiku_meta.cost_input_per_1m}/1M")
print(f"  Output:      ${haiku_meta.cost_output_per_1m}/1M")

In [ ]:
from arcllm.registry import load_model, clear_cache
os.environ.setdefault("ANTHROPIC_API_KEY", "test-key")
os.environ.setdefault("OPENAI_API_KEY", "test-key")

clear_cache()

# load_model() auto-injects pricing from TOML metadata
model = load_model("anthropic", telemetry=True)

print("=== Pricing injected by load_model() ===")
print(f"  Input:       ${model._cost_input}/1M")
print(f"  Output:      ${model._cost_output}/1M")
print(f"  Cache read:  ${model._cost_cache_read}/1M")
print(f"  Cache write: ${model._cost_cache_write}/1M")

In [ ]:
clear_cache()

# Different model → different pricing
model_haiku = load_model("anthropic", "claude-haiku-4-5-20251001", telemetry=True)
print("=== Haiku pricing (auto-injected) ===")
print(f"  Input:  ${model_haiku._cost_input}/1M")
print(f"  Output: ${model_haiku._cost_output}/1M")

In [ ]:
clear_cache()

# Explicit override wins over TOML metadata (setdefault behavior)
model_override = load_model("anthropic", telemetry={"cost_input_per_1m": 99.0})
print("=== Explicit override ===")
print(f"  Input:  ${model_override._cost_input}/1M (override: 99.0)")
print(f"  Output: ${model_override._cost_output}/1M (from TOML metadata)")
print("\nsetdefault() only fills MISSING keys — explicit values are preserved")

The injection code in `registry.py`:
```python
model_meta = config.models.get(model_name)
if model_meta is not None:
    telemetry_config.setdefault("cost_input_per_1m", model_meta.cost_input_per_1m)
    telemetry_config.setdefault("cost_output_per_1m", model_meta.cost_output_per_1m)
    telemetry_config.setdefault("cost_cache_read_per_1m", model_meta.cost_cache_read_per_1m)
    telemetry_config.setdefault("cost_cache_write_per_1m", model_meta.cost_cache_write_per_1m)
```

`setdefault()` only sets if the key doesn't already exist — so explicit overrides in the kwarg dict always win.

---
## 10. Full Module Stack

With all modules enabled: `Telemetry(Retry(Fallback(RateLimit(Adapter))))`

In [ ]:
clear_cache()

model = load_model(
    "anthropic",
    telemetry=True,
    retry=True,
    fallback=True,
    rate_limit=True,
)

# Walk the stack
layer = model
depth = 0
while hasattr(layer, '_inner'):
    indent = "  " * depth
    print(f"{indent}{type(layer).__name__}")
    layer = layer._inner
    depth += 1
print(f"{'  ' * depth}{type(layer).__name__}")

print(f"\n5 layers total — Telemetry measures everything below it")

Why Telemetry is outermost:

```
Telemetry (measures total wall-clock, including:)
  └─ Retry (adds retry wait time)
      └─ Fallback (adds fallback attempt time)
          └─ RateLimit (adds throttle wait time)
              └─ Adapter (actual API call time)
```

The duration_ms you see in the log is the **total time from the agent's perspective** — the most useful operational metric.

---
## 11. Implementation Details

In [ ]:
import inspect

print("=== TelemetryModule.invoke ===")
print(inspect.getsource(TelemetryModule.invoke))

In [ ]:
print("=== TelemetryModule._calculate_cost ===")
print(inspect.getsource(TelemetryModule._calculate_cost))

---
## 12. config.toml Integration

In [ ]:
from arcllm.config import load_global_config

global_config = load_global_config()
tel_config = global_config.modules.get("telemetry")

print("=== config.toml [modules.telemetry] ===")
print(f"  enabled:   {tel_config.enabled}")
print(f"  log_level: {tel_config.model_dump().get('log_level', 'not set')}")
print()
print("When enabled=true in config.toml, telemetry activates by default.")
print("kwarg=False can override to disable. kwarg=True forces enable.")

---
## 13. Live API Call with Telemetry

Real Anthropic API call with `telemetry=True`.

In [ ]:
from dotenv import load_dotenv
load_dotenv(os.path.join(os.getcwd(), '..', '.env'))

has_key = bool(os.environ.get("ANTHROPIC_API_KEY")) and os.environ.get("ANTHROPIC_API_KEY") != "test-key"
print(f"Anthropic API key: {'found' if has_key else 'not found (skip live tests)'}")

In [ ]:
if has_key:
    clear_cache()
    
    # Set up logging so we can see telemetry output
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(levelname)s %(name)s: %(message)s"))
    tel_logger = logging.getLogger("arcllm.modules.telemetry")
    tel_logger.addHandler(handler)
    tel_logger.setLevel(logging.INFO)
    
    model = load_model("anthropic", telemetry=True)
    print(f"Stack: {type(model).__name__}({type(model._inner).__name__})")
    print(f"Pricing: ${model._cost_input}/1M input, ${model._cost_output}/1M output")
    print()
    
    response = await model.invoke([
        Message(role="user", content="What is 2+2? Just the number.")
    ])
    
    print(f"\nResponse: {response.content}")
    
    tel_logger.removeHandler(handler)
    await model._inner.close()
else:
    print("Skipped — no API key")

In [ ]:
if has_key:
    clear_cache()
    
    # Haiku model — different pricing
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(levelname)s %(name)s: %(message)s"))
    tel_logger = logging.getLogger("arcllm.modules.telemetry")
    tel_logger.addHandler(handler)
    tel_logger.setLevel(logging.INFO)
    
    model = load_model("anthropic", "claude-haiku-4-5-20251001", telemetry=True)
    print(f"Haiku pricing: ${model._cost_input}/1M input, ${model._cost_output}/1M output")
    print()
    
    response = await model.invoke([
        Message(role="user", content="What is the speed of light? One sentence.")
    ])
    
    print(f"\nResponse: {response.content}")
    print(f"(Note: Haiku cost per call should be ~4x cheaper than Sonnet)")
    
    tel_logger.removeHandler(handler)
    await model._inner.close()
else:
    print("Skipped — no API key")

In [ ]:
if has_key:
    clear_cache()
    
    # Full stack: telemetry + retry + rate_limit
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(levelname)s %(name)s: %(message)s"))
    tel_logger = logging.getLogger("arcllm.modules.telemetry")
    tel_logger.addHandler(handler)
    tel_logger.setLevel(logging.INFO)
    
    model = load_model("anthropic", telemetry=True, retry=True, rate_limit=True)
    
    # Walk the stack
    layer = model
    layers = []
    while hasattr(layer, '_inner'):
        layers.append(type(layer).__name__)
        layer = layer._inner
    layers.append(type(layer).__name__)
    print(f"Stack: {' → '.join(layers)}")
    print()
    
    response = await model.invoke([
        Message(role="user", content="Say 'telemetry works' in exactly those words.")
    ])
    
    print(f"\nResponse: {response.content}")
    
    tel_logger.removeHandler(handler)
    # Close the innermost adapter
    innermost = model
    while hasattr(innermost, '_inner'):
        innermost = innermost._inner
    await innermost.close()
else:
    print("Skipped — no API key")

---
## Summary

| Component | What | Why |
|-----------|------|-----|
| `TelemetryModule` | Logs timing + tokens + cost per `invoke()` | Operational visibility for thousands of agents |
| `_calculate_cost()` | `(tokens × cost_per_1M) / 1,000,000` | Per-call cost tracking |
| Pricing injection | `load_model()` uses `setdefault()` from provider TOML | Pricing lives in TOML, module doesn't know about ProviderConfig |
| Conditional fields | Cache tokens omitted when None | Reduces log noise |
| `log_level` | Configurable (default INFO) | Toggle verbosity per environment |
| Stack position | Outermost | Measures total wall-clock including retries/fallback/rate-limit |

**Config**:
```python
load_model("anthropic", telemetry=True)                              # Auto-pricing from TOML
load_model("anthropic", telemetry={"log_level": "DEBUG"})            # Custom log level
load_model("anthropic", telemetry={"cost_input_per_1m": 99.0})      # Override pricing
load_model("anthropic", telemetry=False)                             # Disable
```

**Log output**:
```
INFO  arcllm.modules.telemetry: LLM call | provider=anthropic model=claude-sonnet-4-20250514
  duration_ms=1234.5 input_tokens=100 output_tokens=50 total_tokens=150
  cost_usd=0.001050 stop_reason=end_turn
```

**Test count**: 272 passed, 1 skipped (21 new telemetry tests + 7 new registry tests)